In [86]:
import numpy as np
import random
import yfinance as yf
import pandas as pd
import math as m
from scipy.stats import norm

def getMeanandVolatility(ticker, startdate, enddate):
    stock = yf.download(tickers = ticker, start = startdate, end = enddate, prepost = True,  progress=False)
    close = stock["Close"]
    returns=[]
    for i in range(0, len(close)-1):
        tomorrow = close[i+1]
        today = close[i]  
        daily_return = (tomorrow - today)/today
        returns.append(daily_return)
    mu = np.mean(returns)*250
    sigma = np.std(returns)*np.sqrt(250)
    return [mu, sigma]



def EMM_model(S0,mu, sigma, T, steps):
    h = T/steps
    prices=[]
    t = np.arange(0, T, h)
    y = S0
    for j in range(0, t.size):
        y = y + y * mu * h + y * sigma * random.gauss(0, np.sqrt(h))
        prices.append(y)
    return  prices



def BOP_American_model(S0,K,r,delta,sig,T, N,option):
    h = float(T) / N 
    u = np.exp((r-delta)*h + sig * np.sqrt(h)) #up multiplier
    d = np.exp((r-delta)*h - sig * np.sqrt(h)) #down multiplier
    erh = np.exp(r * h)
    p = (erh - d) / (u - d)  # risk neutral up prob
    q = 1.0 - p  # risk neutral down prob
    discount = np.exp(-r*h)
    
    S_T = S0 * d **(np.arange(N,-1,-1)) * u **(np.arange(0,N+1,1))

    #Payoff
    if option == "call":
        Price = np.maximum(S_T - K, 0.0)
    elif option == "put":
        Price = np.maximum(K - S_T, 0.0)

    
    #backward recursion
    for i in range(N - 1, -1, -1):
        S_T = S0 * d **(np.arange(i,-1,-1)) * u **(np.arange(0,i+1,1))
        Price[:i+1] = discount * (p*Price[1:i+2]+q*Price[0:i+1])
        Price = Price[:-1]
        if option == "call":
            Price = np.maximum(S_T - K, Price)
        elif option == "put":
            Price = np.maximum(K - S_T, Price)
    return Price[0]


def Monte_Carlo_Sim(S0,K, r,div,sigma,T, steps, numSim):
    np.random.seed(1)
    sim_steps = steps*T
    h = T/steps
    St = np.zeros(shape=(sim_steps,numSim))
    St[0,] = S0
    for i in range(1,sim_steps):
        for j in range(0,numSim):
            z = np.random.randn(1)
            St[i,j] = St[i-1,j]*m.exp((r-div-0.5*sigma*sigma)*h+sigma*m.sqrt(h)*z)
    sim_stocks = pd.DataFrame(St)
    prices = []
    Y = []
    for i in range(0,sim_steps):
        prices.append(np.mean(sim_stocks[i])) # Taking the arithmetic mean
        Y.append(max(prices[i]-K,0)*m.exp(-r*T))
    arithmetic_opt_price  = np.mean(Y)
    
    return arithmetic_opt_price



def BlackScholes (S, K, r, delta, sigma, T, Option):
    d1 = (m.log(S/K) + (r-delta+.5*sigma*sigma)*T)/(sigma*m.sqrt(T))
    d2 = d1 - sigma*m.sqrt(T)
    if Option=="call":
        result = S*m.exp(-1*delta*T)*norm.cdf(d1) - K*m.exp(-1*r*T)*norm.cdf(d2)
        return result
    else:
        result = K * m.exp(-1*r*T)*norm.cdf(-1*d2) - S*m.exp(-1*delta*T)*norm.cdf(-1*d1)
        return result
    

